In [1]:
import cobra

In [2]:
model = cobra.io.load_matlab_model("../data/HMA_constrained/HMR3674wFluxes_786-O_FBS.mat")
cobra.io.write_sbml_model(model,"../data/HMA_constrained/HMR3674wFluxes_786-O_FBS.xml")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jeremiebourdon/miniconda3/envs/Bravo_Experiments/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-b96a6f36180e>", line 1, in <module>
    model = cobra.io.load_matlab_model("../data/HMA_constrained/HMR3674wFluxes_786-O_FBS.mat")
  File "/Users/jeremiebourdon/miniconda3/envs/Bravo_Experiments/lib/python3.6/site-packages/cobra/io/mat.py", line 82, in load_matlab_model
    inf=inf)
  File "/Users/jeremiebourdon/miniconda3/envs/Bravo_Experiments/lib/python3.6/site-packages/cobra/io/mat.py", line 259, in from_mat_struct
    model.reactions[j].add_metabolites({model.metabolites[i]: v})
  File "/Users/jeremiebourdon/miniconda3/envs/Bravo_Experiments/lib/python3.6/site-packages/cobra/core/reaction.py", line 785, in add_metabolites
    self.reverse_variable: -coefficient
  File "/Users/jeremiebourdon/miniconda3/envs/Bravo_Experimen

KeyboardInterrupt: 

In [ ]:
model.optimize()

In [6]:
for r in model.reactions:
    if (r.objective_coefficient == 1):
        print(r.name, r.objective_coefficient)

In [11]:
print(len(model.objective))

0


In [12]:
filename = "../data/HMA_constrained/HMR3674wFluxes_786-O_FBS.mat"


In [14]:
filename.count(".xml")

0

In [17]:
model.reactions.EXC_IN_m01451c.build_reaction_string()

' <=> m01451c'

In [19]:
model.genes.ENSG00000000419.reactions

frozenset({<Reaction HMR_7271 at 0x123d51710>})

In [13]:
# coding: utf-8

# # Un scénario complet

# Quelques informations et paramètres pour commencer

# In[1]:

from Entete import *


# Tous les includes

# In[2]:

#Param_SBMLfile = "/Users/jeremiebourdon/Downloads/Hep-G2.xml"


# In[3]:

## DEBUG ONLY

Param_SBMLfile = "../data/HMA_constrained/HMR3674wFluxes_786-O_FBS.mat"

begintotal = timer()
eprint("Testing",Param_SBMLfile,"for",Param_Drug)

## END DEBUG ONLY


# Chargement des listes de correspondances

# In[5]:

#from AllTables import *
from AllTables import *


# In[ ]:




# In[6]:

Param_Pourcentage = 0.95


# Construction de la liste des cibles

# In[7]:

# étape 1 : construction du fichier d'association Reactions <-> EC numbers

# obsolete : La liste RXN -> liste gènes est désormais récupérée directement depuis HMRDatabase

# dictionnaire HMR_to_Ensembl


eprint("Entering : Association RxN <-> Enzymes list construction")

SBMLfile = Param_SBMLfile

#SBMLfile = raw_input("Enter the SBML filename ? ")

def ConstructAssociationList(SBMLfile, RXN_EC_file="/tmp/RXN_EC.txt"):

    import os
    os.system("../scripts/Construct_List_RXN_Enzymes.sh "+SBMLfile+" "+RXN_EC_file)
    fd = open(RXN_EC_file,"r")
    lines = fd.readlines()

    genes = dict()
    for l in lines:
        t=l.split(" ")
        cum=''
        for i in range(1,len(t)-1):
            cum=cum+t[i]+" "
        genes[t[0]]=cum
    return genes

# genes=ConstructAssociationList(SBMLfile,RXN_EC_file)


# In[8]:

# Etape 2 : Single Reaction Analysis

eprint("Entering : Single Reaction Analysis")

## DEBUG ONLY
begin = timer()
## END DEBUG ONLY


# perfoming a single_reaction_deletion analysis of the network

if (Param_SBMLfile.count(".xml")): # is it an xml or a .mat ?
    model = cobra.io.read_sbml_model(Param_SBMLfile)
else:
    model = cobra.io.load_matlab_model(Param_SBMLfile)

# the objective coefficient must be properly set
# try to add an objective for HMA_r3 sbml files
#if (len(model.objective) == 0):
#    model.reactions.CancerBiomass_OF.objective_coefficient=1

# reference biomass growth
fba=model.optimize().f

# single reaction deletion analysis


if (Param_DoReconstruction and Param_Pourcentage<1):
    ratesD = cobra.flux_analysis.single_reaction_deletion(model) #,solver=whichsolver)
    ratesD = ratesD.get('growth').to_dict() # for cobra 0.11.3
    rates=dict()                            # for cobra 0.11.3
    for r in ratesD:                        # for cobra 0.11.3
        rates[list(r)[0]]=ratesD.get(r)

else:
    rates=dict()
    for r in model.reactions:
        rates[r.id]=0

## DEBUG ONLY
end = timer()
eprint("Execution time (Single Reaction Analysis) = ",end-begin,"seconds")
## END DEBUG ONLY


# In[ ]:




# In[ ]:




# In[ ]:




# In[9]:

# Etape 3 : Extraction des enzymes régulateurs (nécessite étape 1 et 2)

def getRegulators(rates, percentage=1.1):
    regulators = dict()

    cpt=0
    for r in rates:
        if (rates.get(r)<percentage*fba):
            if (HMR_to_Ensembl.get(r)!=''):
                # print(r,':',rates.get(r)," ",genes.get(r))
                cpt=cpt+1
                l=HMR_to_Ensembl.get(r)
                if (l):
                    for i in l:
                        if (i != ''):
                            regulators[i]=1
    return regulators


def getRegulators_for_constrained_models(rates, percentage=1.1):
    regulators = dict()

    cpt=0
    for r in rates:
        if (rates.get(r)<percentage*fba):
            react=model.reactions.get_by_id(r)
            for g in react.genes:
                regulators[g]=1
    return regulators



def getRegulators_obsolete(rates, percentage=1.1):
    regulators = dict()

    cpt=0
    for r in rates:
        if (rates.get(r)<percentage*fba):
            if (genes.get(r)!=''):
                # print(r,':',rates.get(r)," ",genes.get(r))
                cpt=cpt+1
                l=genes.get(r).split(' ')
                for i in l:
                    if (i != ''):
                        regulators[i]=1
    return regulators

Testing ../data/HMA_constrained/HMR3674wFluxes_786-O_FBS.mat for Sorafenib (USAN/INN)
Entering : Association RxN <-> Enzymes list construction
Entering : Single Reaction Analysis
Execution time (Single Reaction Analysis) =  20.86535510106478 seconds


In [16]:
if (Param_DoReconstruction):
    regulators = getRegulators_for_constrained_models(rates, Param_Pourcentage)
    #regulators = getRegulators(rates, Param_Pourcentage) # for HMA_r3

In [17]:
regulators

{}

In [19]:
model=cobra.io.read_sbml_model("../data/HMA_constrained/HMR3674wFluxes_786-O_FBS.xml")
model.optimize()

,fluxes,reduced_costs
HMR_3905,0.0000,-4.100509e-16
HMR_3907,0.0000,-4.450052e-17
HMR_4097,0.0000,-1.323494e-15
HMR_4099,0.0000,-7.419477e-16
HMR_4108,0.0000,-1.323494e-15
...,...,...
EXC_IN_m02949c,0.0000,8.935195e-17
EXC_IN_m02965c,0.0000,-1.162173e-14
EXC_IN_m02969c,0.0000,-2.301581e-16
EXC_IN_m02998c,0.0000,-2.378835e-16


In [20]:
ratesD = cobra.flux_analysis.single_reaction_deletion(model)

In [21]:
ratesD = ratesD.get('growth').to_dict() # for cobra 0.11.3
rates=dict()                            # for cobra 0.11.3
for r in ratesD:                        # for cobra 0.11.3
    rates[list(r)[0]]=ratesD.get(r)


In [22]:
model.optimize()

,fluxes,reduced_costs
HMR_3905,0.0000,-4.100509e-16
HMR_3907,0.0000,-4.450052e-17
HMR_4097,0.0000,-1.323494e-15
HMR_4099,0.0000,-7.419477e-16
HMR_4108,0.0000,-1.323494e-15
...,...,...
EXC_IN_m02949c,0.0000,8.935195e-17
EXC_IN_m02965c,0.0000,-1.162173e-14
EXC_IN_m02969c,0.0000,-2.301581e-16
EXC_IN_m02998c,0.0000,-2.378835e-16


In [23]:
rates

{'HMR_8825': 85.91687794269252,
 'HMR_8815': 85.91687794269252,
 'HMR_1263': 85.91687794269254,
 'HMR_2868': 85.91687794269254,
 'HMR_7857': 85.91687794269467,
 'HMR_6178': 85.91687794269467,
 'HMR_5315': 85.91687794269467,
 'HMR_4990': 85.9168779426928,
 'HMR_4682': 85.91687794269241,
 'HMR_1037': 85.91687794269241,
 'HMR_7416': 85.91687794269241,
 'HMR_4540': 85.91687794269241,
 'HMR_8663': 85.91687794269241,
 'HMR_9399': 85.91687794269241,
 'HMR_5618': 85.91687794269241,
 'HMR_7452': 85.91687794269241,
 'HMR_3333': 85.91687794269241,
 'HMR_3130': 85.9168779426921,
 'HMR_5748': 85.9168779426921,
 'HMR_2609': 85.9168779426921,
 'HMR_4423': 85.9168779426921,
 'HMR_7254': 85.9168779426921,
 'HMR_4143': 85.91687794269238,
 'HMR_7714': 85.91687794269238,
 'HMR_3591': 85.91687794269238,
 'HMR_9710': 85.91687794269238,
 'HMR_3521': 85.91687794269238,
 'HMR_0341': 85.91687794270993,
 'HMR_7787': 85.91687794269238,
 'HMR_5877': 85.91687794269247,
 'HMR_8725': 85.91687794269247,
 'HMR_9612': 8

In [39]:
import math
for r in rates:
    if (math.isnan(rates.get(r)) or rates.get(r)<85):
        react = model.reactions.get_by_id(r)
        l=[g.id for g in list(react.genes)]
        print(r, rates.get(r), l)
        
        


HMR_0498 nan ['ENSG00000186281', 'ENSG00000158669', 'ENSG00000119927', 'ENSG00000138678']
RenalCancerBOF_20130312 3.1546850921947874e-19 []
HMR_3877 nan ['ENSG00000101444', 'ENSG00000168710', 'ENSG00000158467']
HMR_4050 nan ['ENSG00000084774']
HMR_4810 6.76013043498406 ['ENSG00000128050']
HMR_0532 nan ['ENSG00000186281', 'ENSG00000158669', 'ENSG00000119927', 'ENSG00000138678']
HMR_3785 nan ['ENSG00000138029', 'ENSG00000084754', 'ENSG00000127884', 'ENSG00000113790']
HMR_3773 77.85048445694146 ['ENSG00000078070', 'ENSG00000131844']
HMR_9045 -1.9436307238459612e-13 []
HMR_9064 67.9285002882136 []
HMR_0501 nan ['ENSG00000186281', 'ENSG00000158669', 'ENSG00000119927', 'ENSG00000138678']
HMR_0523 nan ['ENSG00000186281', 'ENSG00000158669', 'ENSG00000119927', 'ENSG00000138678']
HMR_0512 nan ['ENSG00000186281', 'ENSG00000158669', 'ENSG00000119927', 'ENSG00000138678']
HMR_4644 0.7406697450417387 ['ENSG00000176890']
HMR_9040 9.72196039104922e-15 []
HMR_0496 nan ['ENSG00000186281', 'ENSG0000015866

In [40]:
def getRegulators_for_constrained_models(rates, percentage=1.1):
    regulators = dict()

    cpt=0
    for r in rates:
        if (math.isnan(rates.get(r)) or rates.get(r)<percentage*fba):
            react=model.reactions.get_by_id(r)
            l=[g.id for g in list(react.genes)]
            for g in l:
                regulators[g]=1
    return regulators


In [41]:
getRegulators_for_constrained_models(rates,0.95)

{'ENSG00000011009': 1,
 'ENSG00000011052': 1,
 'ENSG00000014257': 1,
 'ENSG00000032444': 1,
 'ENSG00000038274': 1,
 'ENSG00000054179': 1,
 'ENSG00000062282': 1,
 'ENSG00000068366': 1,
 'ENSG00000072506': 1,
 'ENSG00000075239': 1,
 'ENSG00000076685': 1,
 'ENSG00000084754': 1,
 'ENSG00000084774': 1,
 'ENSG00000092964': 1,
 'ENSG00000095139': 1,
 'ENSG00000099194': 1,
 'ENSG00000101438': 1,
 'ENSG00000101444': 1,
 'ENSG00000102967': 1,
 'ENSG00000103024': 1,
 'ENSG00000103066': 1,
 'ENSG00000103202': 1,
 'ENSG00000103502': 1,
 'ENSG00000103740': 1,
 'ENSG00000104331': 1,
 'ENSG00000104812': 1,
 'ENSG00000105198': 1,
 'ENSG00000105205': 1,
 'ENSG00000105669': 1,
 'ENSG00000106384': 1,
 'ENSG00000107902': 1,
 'ENSG00000111713': 1,
 'ENSG00000112981': 1,
 'ENSG00000113657': 1,
 'ENSG00000113790': 1,
 'ENSG00000115705': 1,
 'ENSG00000116711': 1,
 'ENSG00000116761': 1,
 'ENSG00000116981': 1,
 'ENSG00000116984': 1,
 'ENSG00000119927': 1,
 'ENSG00000120992': 1,
 'ENSG00000122218': 1,
 'ENSG00000

In [82]:
model = cobra.io.read_sbml_model("/Users/jeremiebourdon/Downloads/iBth801 v1.00.xml")

In [65]:
print(model.reactions.biomass_components)

biomass_components: m1637 + m1703 + m1709 + m1712 + m1833 + m1841 + m2069 + m2095 + m2122 + m2263 + m2264 + m2565 + m2570 + m2585 + m2802 + m2867 + m3050 + m3104 + m3148 + m3199 + m3496 + m3508 + m3529 + m3557 + m3638 + m3686 + m3701 + m3850 + m3978 + m3983 + m4050 + m4080 --> 


In [66]:
for met in model.reactions.biomass_components.metabolites:
    newid = met.id.replace('c','s')
    if (model.metabolites.has_id(newid)):
        m=model.metabolites.get_by_id(newid)
        for r in m.reactions:
            #print(m,r)
            if (len(r.metabolites) == 1):
                print(m,m.name, r.id, r.name, r.build_reaction_string(), r.lower_bound, r.upper_bound)

In [74]:
model.reactions.biomass_components.objective_coefficient=1

In [83]:
fba = model.optimize()

In [84]:
fba.f

0.0

In [70]:
cobra.checkObjective(model)

AttributeError: module 'cobra' has no attribute 'checkObjective'

In [85]:
for r in model.reactions:
    if (r.objective_coefficient>0):
        print(r)

In [78]:
cobra.flux_analysis.find_essential_genes(model)

set()

In [86]:
model.reactions.biomassTrans.objective_coefficient = 1

In [91]:
for r in model.reactions:
    if len(r.genes)>1:
        print(r)

In [92]:
model = cobra.io.read_sbml_model("../data/HMA_PersonalizedModels/liver cancer - patient 2177.xml")

In [95]:
for r in model.reactions:
    if r.id.count("bio")>0:
        print(r.id)

biomass_components


In [96]:
model.repair()

In [99]:
for r in model.reactions:
    if len(r.metabolites)==1:
        print(r, r.metabolites, r.lower_bound, r.upper_bound)

HMR_7108:  <=> m1727 {<Metabolite m1727 at 0x11bc2f668>: 1.0} -1000 1000
HMR_7110:  <=> m3285 {<Metabolite m3285 at 0x11e5eacf8>: 1.0} -1000 1000
HMR_7112:  <=> m1623 {<Metabolite m1623 at 0x11b1e7d68>: 1.0} -1000 1000
HMR_7114:  <=> m3929 {<Metabolite m3929 at 0x11b428da0>: 1.0} -1000 1000
HMR_7116:  <=> m1762 {<Metabolite m1762 at 0x11d286fd0>: 1.0} -1000 1000
HMR_7118:  <=> m1476 {<Metabolite m1476 at 0x11b23d390>: 1.0} -1000 1000
HMR_7120:  <=> m1200 {<Metabolite m1200 at 0x128137518>: 1.0} -1000 1000
HMR_7122:  <=> m735 {<Metabolite m735 at 0x11b19e550>: 1.0} -1000 1000
HMR_7124:  <=> m378 {<Metabolite m378 at 0x11b0acac8>: 1.0} -1000 1000
HMR_7126:  <=> m393 {<Metabolite m393 at 0x11b0a2b00>: 1.0} -1000 1000
HMR_9024:  --> m2035 {<Metabolite m2035 at 0x11ea4ff60>: 1.0} 0 1000
HMR_9025:  --> m4062 {<Metabolite m4062 at 0x11b275ba8>: 1.0} 0 1000
HMR_9028:  --> m2034 {<Metabolite m2034 at 0x129cf7518>: 1.0} 0 1000
HMR_9029:  --> m4061 {<Metabolite m4061 at 0x11b275cf8>: 1.0} 0 1000


In [100]:
model.objective = model.reactions.biomass_components.flux_expression

In [101]:
model.optimize()

,fluxes,reduced_costs
HMR_3905,0.0,0.0
HMR_3907,0.0,0.0
HMR_4099,0.0,0.0
HMR_4137,0.0,0.0
HMR_4281,0.0,0.0
...,...,...
HMR_9693,0.0,0.0
HMR_9715,0.0,-0.0
HMR_9730,0.0,-0.0
HMR_9736,0.0,0.0
